## 2.- Sizing Code Evolutionary Algortithm

In [1]:
import scipy
import scipy.optimize
from math import pi
from math import sqrt
import math
import timeit
import time

import numpy as np

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display
import pandas as pd


In [2]:
# Algorithm to find the lower, upper, closest reference from a catalogue given some optimization data

# lowerRef() will return the catalogue reference inmediately below from the input given.

def lowerRef(str1,var1,str2,var2,pro_arg,df):
    myElemList = df[str1] #table is filtered according to the first parameter
    lowerVar1= min(myElemList, key=lambda x:(abs(x-var1) if x < var1 else 1e10)) #find the closest reference smaller than the desired reference
    df_var1=df.loc[(myElemList == lowerVar1)] #filter data containing such values
    myElem2List = df_var1[str2] #filter diameter from the beta subset
    
    #the second parameter is classified according to the criteria:
    if pro_arg=='Find a lower ref':
        lowerVar2= min(myElem2List, key=lambda x:(abs(x-var2) if x < var2 else 1e10)) #find the value in the datasheet closest to the desired reference
        model=df.loc[(df[str1]==lowerVar1) & (df[str2]==lowerVar2)].Model #select the type/model of the component 
        return lowerVar1,lowerVar2,model
    elif pro_arg=='Find the closest ref':
        closestVar2= min(myElem2List, key=lambda x:abs(x-var2)) #find the value in the subset closest to the desired reference
        model=df.loc[(df[str1]==lowerVar1) & (df[str2]==closestVar2)].Model #select the type/model of the component 
        return lowerVar1,closestVar2,model
    else  :
        upperVar2= min(myElem2List, key=lambda x:(abs(x-var2) if x > var2 else 1e10)) #find the value in the datasheet closest to the desired reference
        model=df.loc[(df[str1]==lowerVar1) & (df[str2]==upperVar2)].Model #select the type/model of the component 
        return lowerVar1,upperVar2,model

# upperRef() will return the catalogue reference inmediately above from the input given.
    
def upperRef(str1,var1,str2,var2,pro_arg,df):
    myElemList = df[str1]#table is filtered according to the first parameter
    upperVar1= min(myElemList, key=lambda x:(abs(x-var1) if x > var1 else 1e10)) #find the value in the datasheet closest to the desired reference
    df_var1=df.loc[(myElemList == upperVar1)] #filter danta containing such values
    myElem2List = df_var1[str2] #filter diameter from the beta subset

    #the second parameter is classified according to the criteria:
    if pro_arg=='Find a lower ref':
        lowerVar2= min(myElem2List, key=lambda x:(abs(x-var2) if x < var2 else 1e10)) #find the value in the datasheet closest to the desired reference
        model=df.loc[(df[str1]==upperVar1) & (df[str2]==lowerVar2)].Model #select the type/model of the component 
        return upperVar1,lowerVar2,model
    elif pro_arg=='Find the closest ref':
        closestVar2= min(myElem2List, key=lambda x:abs(x-var2)) #find the value in the subset closest to the desired reference
        model=df.loc[(df[str1]==upperVar1) & (df[str2]==closestVar2)].Model #select the type/model of the component 
        return upperVar1,closestVar2,model
    else  :
        upperVar2= min(myElem2List, key=lambda x:(abs(x-var2) if x > var2 else 1e10)) #find the value in the datasheet closest to the desired reference
        model=df.loc[(df[str1]==upperVar1) & (df[str2]==upperVar2)].Model #select the type/model of the component 
        return upperVar1,upperVar2,model

# closeRef() will return the closest catalogue reference from the input given.

def closeRef(str1,var1,str2,var2,pro_arg,df):
    myElemList = df[str1]
    closeVar1= min(myElemList, key=lambda x: abs(x-var1)) #find the value in the datasheet closest to the desired reference
    df_var1=df.loc[(myElemList == closeVar1)] #create dataframe containing such values
    myElem2List = df_var1[str2] #filter diameter from the beta subset
    
    #the second parameter is classified according to the criteria:

    if pro_arg=='Find a lower ref':
        lowerVar2= min(myElem2List, key=lambda x:(abs(x-var2) if x < var2 else 1e10)) #find the value in the datasheet closest to the desired reference
        model=df.loc[(df[str1]==closeVar1) & (df[str2]==lowerVar2)].Model #select the type/model of the component 
        return closeVar1,lowerVar2,model
    elif pro_arg=='Find the closest ref':
        closestVar2= min(myElem2List, key=lambda x:abs(x-var2)) #find the value in the subset closest to the desired reference
        model=df.loc[(df[str1]==closeVar1) & (df[str2]==closestVar2)].Model #select the type/model of the component 
        return closeVar1,closestVar2,model
    else  :
        upperVar2= min(myElem2List, key=lambda x:(abs(x-var2) if x > var2 else 1e10)) #find the value in the datasheet closest to the desired reference
        model=df.loc[(df[str1]==closeVar1) & (df[str2]==upperVar2)].Model #select the type/model of the component 
        return closeVar1,upperVar2,model

#### 2.- Problem Definition

In [3]:
#1st column data definition
style = {'description_width': '150pt'}
layout = {'width': 'initial'}


specs = widgets.HTML(value = f"<b><font color='red'>{'Specifications:'}</b>")
M_load = widgets.Text(value='',placeholder='e.g. 1',description='Load mass [kg]',disabled=False,style=style, layout=layout)
t_h = widgets.Text(value='',placeholder='e.g. 10',description='Hover flight time [min]',disabled=False,style=style, layout=layout)

MTOW = widgets.Text(value='',placeholder='e.g. 3',description='[kg] maximal mass',disabled=False,style=style, layout=layout)
k_maxthrust = widgets.Text(value='',placeholder='e.g. 3',description='Ratio max thrust-hover [-]',disabled=False,style=style, layout=layout)
rho_air = widgets.Text(value='',placeholder='e.g. 1.18',description='[kg/m^3] air density ',disabled=False,style=style, layout=layout)
                     

archit = widgets.HTML(value = f"<b><font color='red'>{'Architecture'}</b>")


Narm = widgets.Text(value='',placeholder='e.g. 8',description='number of arms [-]',disabled=False,style=style, layout=layout)
Np_arm = widgets.Text(value='',placeholder='e.g. 1',description='number of propeller per arm',disabled=False,style=style, layout=layout)
Mod=widgets.ToggleButtons(options=['Direct Drive', 'Gear Drive'],
    description='Motor configuration:',
    #disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['No speed reductor', 'Motor with reduction', 'Description of fast'],
    style=style,
    layout=layout
    )

vertical = widgets.HTML(value = f"<b><font color='red'>{'Vertical climb'}</b>")

V_cl = widgets.Text(value='',placeholder='e.g. 1',description='max climb speed [m/s]',disabled=False,style=style, layout=layout)
C_D= widgets.Text(value='',placeholder='e.g. 1.3',description='drag coef [-]',disabled=False,style=style, layout=layout)
A_top= widgets.Text(value='',placeholder='e.g. 0.05',description='[m^2] top surface',disabled=False,style=style, layout=layout)
  
prop = widgets.HTML(value = f"<b><font color='red'>{'Propeller characteristics'}</b>")

NDmax= widgets.Text(value='',placeholder='e.g. 105000',description='[RPM*in] RPM limit',disabled=False,style=style, layout=layout)

objectif =widgets.HTML(value = f"<b><font color='red'>{'Objective and optimization'}</b>")

Obj=widgets.ToggleButtons(options=['Minimize mass', 'Maximize time'],
    description='Objective:',
    disabled=False,
    style=style,
    layout=layout
    )
                       
Algo=widgets.ToggleButtons(options=['SLSQP', 'Differential evolution'],
    description='Resolution algorithm:',
    disabled=False,
    style=style,
    layout=layout
    )

path='./Propeller/'
df_pro = pd.read_csv(path+'Non-Dominated-Propeller.csv', sep=';')
path='./Batteries/'
df_bat = pd.read_csv(path+'Non-Dominated-Batteries.csv', sep=';')
path='./Motors/'
df_mot = pd.read_csv(path+'Non-Dominated-Motors.csv', sep=';')
path='./ESC/'
df_esc = pd.read_csv(path+'Non-Dominated-ESC.csv', sep=';')

                       
Catalogue=widgets.HTML(value = f"<b><font color='red'>{'Catalogue selection:'}</b>")

options=['Find a lower ref', 'Find the closest ref', 'Find an upper ref']
             
C1 = widgets.ToggleButtons(options=options, style=style, layout=layout, description='Propeller diameter [in]:')
C2 = widgets.ToggleButtons(options=options, style=style, layout=layout, description='ratio pitch-to-diameter [-]:')
C3 = widgets.ToggleButtons(options=options, style=style, layout=layout, description='Nominal torque [N.m]:')
C4 = widgets.ToggleButtons(options=options, style=style, layout=layout, description='Kt constant [N.m/A]:')
C5 = widgets.ToggleButtons(options=options, style=style, layout=layout, description='ESC power [W]:')
C6 = widgets.ToggleButtons(options=options, style=style, layout=layout, description='ESC voltage [V]:')
C7 = widgets.ToggleButtons(options=options, style=style, layout=layout, description='Battery capacity [mA.h]:')
C8 = widgets.ToggleButtons(options=options, style=style, layout=layout, description='Battery voltage [V]:')

w1=widgets.VBox([specs,M_load,t_h,MTOW,k_maxthrust,rho_air,archit,Narm,Np_arm,Mod,vertical,V_cl,C_D,A_top,prop,NDmax,objectif,Obj,Algo])
w2=widgets.VBox([Catalogue,C1,C2,C3,C4,C5,C6,C7,C8])
                       
#3rd column choice of the family component


# items_pro = sorted(df_pro['TYPE'].unique().tolist())
# items_bat = sorted(df_bat['TYPE'].unique().tolist())
# items_esc = sorted(df_esc['TYPE'].unique().tolist())
# items_mot = sorted(df_mot['TYPE'].unique().tolist())
 
# pro = widgets.Select(options=items_pro,description='propeller family:',style=style,layout={'width': '350px'})
# bat = widgets.Select(options=items_bat,description='battery family:',style=style,layout={'width': '350px'})
# esc = widgets.Select(options=items_esc,description='ESC family:',style=style,layout={'width': '350px'})
# mot = widgets.Select(options=items_mot,description='motor family:',style=style,layout={'width': '350px'})

ui = widgets.HBox([w1,w2])

# hbox=widgets.HBox([pro, bat, esc, mot])
display(ui,
#         hbox
       )

#### 3.- Sizing Code

In [4]:
# -----------------------
# sizing code
# -----------------------
# inputs:
# - param: optimisation variables vector (reduction ratio, oversizing coefficient)
# - arg: selection of output  
# output: 
# - objective if arg='Obj', problem characteristics if arg='Prt', constraints other else

def SizingCode(param, arg):
# Design variables
# ---
    k_M=param[0] # over sizing coefficient on the load mass 
    k_mot=param[1] # over sizing coefficient on the motor torque
    k_speed_mot=param[2] # over sizing coefficient on the motor speed
    k_vb=param[3] # over sizing coefficient for the battery voltage
    k_ND=param[4] # slow down propeller coef : ND = kNDmax / k_ND
    D_ratio=param[5] # aspect ratio e/c (thickness/side) for the beam of the frame
    k_Mb=param[6] # over sizing coefficient on the battery load mass 
    beta=param[7] # pitch/diameter ratio of the propeller
    J=param[8] # advance ratio 
    k_ESC=param[9] # over sizing coefficient on the ESC power
    if Mod.value=='Gear Drive':
        Nred=param[10]   # Reduction Ratio [-]
# Hover, Climbing & Take-Off thrust 
# ---
    Npro=eval(Np_arm.value)*eval(Narm.value) # [-] Propellers number
    Mtotal=k_M*eval(M_load.value) # [kg] Estimation of the total mass (or equivalent weight of dynamic scenario)
    
    F_pro_hov=Mtotal*(9.81)/Npro # [N] Thrust per propeller for hover
    F_pro_to=F_pro_hov*eval(k_maxthrust.value) # [N] Max Thrust per propeller    
    F_pro_cl=(Mtotal*9.81+0.5*eval(rho_air.value)*eval(C_D.value)*eval(A_top.value)*eval(V_cl.value)**2)/Npro # [N] Thrust per propeller for climbing
    
# Propeller characteristicss
# Ref : APC static

    C_t_sta=4.27e-02 + 1.44e-01 * beta # Thrust coef with T=C_T.rho.n^2.D^4
    C_p_sta=-1.48e-03 + 9.72e-02 * beta  # Power coef with P=C_p.rho.n^3.D^5

    Dpro_ref=11*.0254 # [m] diameter
    Mpro_ref=0.53*0.0283 # [kg] mass
    
# Ref: APC dynamics

    C_t_dyn=0.02791-0.06543*J+0.11867*beta+0.27334*beta**2-0.28852*beta**3+0.02104*J**3-0.23504*J**2+0.18677*beta*J**2 # thrust coef for APC props in dynamics
    C_p_dyn=0.01813-0.06218*beta+0.00343*J+0.35712*beta**2-0.23774*beta**3+0.07549*beta*J-0.1235*J**2 # power coef for APC props in dynamics

    
  #Choice of diameter and rotational speed from a maximum thrust
    Dpro=(F_pro_to/(C_t_sta*eval(rho_air.value)*(eval(NDmax.value)/60*0.0254*k_ND)**2))**0.5  # [m] Propeller diameter

    if C2.value =='Find a lower ref':
        Dpro=39.3701*Dpro #[in] diameter expressed in inches
        beta=lowerRef('DIAMETER',Dpro,'BETA',beta,C1.value,df_pro)[1]
        Dpro=lowerRef('DIAMETER',Dpro,'BETA',beta,C1.value,df_pro)[0]*0.0254#[m] diameter expressed in meters
        C_t_sta=4.27e-02 + 1.44e-01 * beta # Thrust coef with T=C_T.rho.n^2.D^4
        C_p_sta=-1.48e-03 + 9.72e-02 * beta  # Power coef with P=C_p.rho.n^3.D^5
        C_t_dyn=0.02791-0.06543*J+0.11867*beta+0.27334*beta**2-0.28852*beta**3+0.02104*J**3-0.23504*J**2+0.18677*beta*J**2 # thrust coef for APC props in dynamics
        C_p_dyn=0.01813-0.06218*beta+0.00343*J+0.35712*beta**2-0.23774*beta**3+0.07549*beta*J-0.1235*J**2 # power coef for APC props in dynamics
        modelPro=lowerRef('DIAMETER',Dpro,'BETA',beta,C1.value,df_pro)[2].iloc[0]
    elif C2.value =='Find the closest ref':
        Dpro=39.3701*Dpro #[in] diameter expressed in inches
        Dpro=closeRef('DIAMETER',Dpro,'BETA',beta,C1.value,df_pro)[0]*0.0254#[m] diameter expressed in meters
        beta=closeRef('DIAMETER',Dpro,'BETA',beta,C1.value,df_pro)[1]
        C_t_sta=4.27e-02 + 1.44e-01 * beta # Thrust coef with T=C_T.rho.n^2.D^4
        C_p_sta=-1.48e-03 + 9.72e-02 * beta  # Power coef with P=C_p.rho.n^3.D^5
        C_t_dyn=0.02791-0.06543*J+0.11867*beta+0.27334*beta**2-0.28852*beta**3+0.02104*J**3-0.23504*J**2+0.18677*beta*J**2 # thrust coef for APC props in dynamics
        C_p_dyn=0.01813-0.06218*beta+0.00343*J+0.35712*beta**2-0.23774*beta**3+0.07549*beta*J-0.1235*J**2 # power coef for APC props in dynamics
        modelPro=closeRef('DIAMETER',Dpro,'BETA',beta,C1.value,df_pro)[2].iloc[0]

    else:
        Dpro=39.3701*Dpro #[in] diameter expressed in inches
        Dpro=upperRef('DIAMETER',Dpro,'BETA',beta,C1.value,df_pro)[0]*0.0254#[m] diameter expressed in meters
        beta=upperRef('DIAMETER',Dpro,'BETA',beta,C1.value,df_pro)[1]
        C_t_sta=4.27e-02 + 1.44e-01 * beta # Thrust coef with T=C_T.rho.n^2.D^4
        C_p_sta=-1.48e-03 + 9.72e-02 * beta  # Power coef with P=C_p.rho.n^3.D^5
        C_t_dyn=0.02791-0.06543*J+0.11867*beta+0.27334*beta**2-0.28852*beta**3+0.02104*J**3-0.23504*J**2+0.18677*beta*J**2 # thrust coef for APC props in dynamics
        C_p_dyn=0.01813-0.06218*beta+0.00343*J+0.35712*beta**2-0.23774*beta**3+0.07549*beta*J-0.1235*J**2 # power coef for APC props in dynamics
        modelPro=upperRef('DIAMETER',Dpro,'BETA',beta,C1.value,df_pro)[2].iloc[0]

# Propeller selection with take-off scenario
    pitch=beta*Dpro # [m] Propeller pitch
    n_pro_to=eval(NDmax.value)/60*0.0254*k_ND/Dpro # [Hz] Propeller speed 
    n_pro_cl=sqrt(F_pro_cl/(C_t_dyn*eval(rho_air.value)*Dpro**4)) # [Hz] climbing speed
    
    Wpro_to=n_pro_to*2*3.14 # [rad/s] Propeller speed
    Mpro=Mpro_ref*(Dpro/Dpro_ref)**3 # [kg] Propeller mass
    Ppro_to=C_p_sta*eval(rho_air.value)*n_pro_to**3*Dpro**5# [W] Power per propeller
    Qpro_to=Ppro_to/Wpro_to # [N.m] Propeller torque

# Propeller torque& speed for hover

    n_pro_hover=sqrt(F_pro_hov/(C_t_sta*eval(rho_air.value)*Dpro**4)) # [Hz] hover speed
    Wpro_hover=n_pro_hover*2*3.14 # [rad/s] Propeller speed    
    
    Ppro_hover=C_p_sta*eval(rho_air.value)*n_pro_hover**3*Dpro**5# [W] Power per propeller
    Qpro_hover=Ppro_hover/Wpro_hover # [N.m] Propeller torque    

    V_bat_est=k_vb*1.84*(Ppro_to)**(0.36) # [V] battery voltage estimation
    
#Propeller torque &speed for climbing
    Wpro_cl=n_pro_cl*2*3.14 # [rad/s] Propeller speed for climbing   
    
    Ppro_cl=C_p_dyn*eval(rho_air.value)*n_pro_cl**3*Dpro**5# [W] Power per propeller for climbing
    Qpro_cl=Ppro_cl/Wpro_cl # [N.m] Propeller torque for climbing   

    
# Motor selection & scaling laws
# ---
    # Motor reference sized from max thrust
    # Ref : AXI 5325/16 GOLD LINE
    
        
    Tmot_ref=2.32  # [N.m] rated torque
    Tmot_max_ref=85/70*Tmot_ref # [N.m] max torque
    Rmot_ref=0.03  # [Ohm] resistance
    Mmot_ref=0.575 # [kg] mass
    Ktmot_ref=0.03 # [N.m/A] torque coefficient
    Tfmot_ref=0.03 # [N.m] friction torque (zero load, nominal speed) 

    #Motor speeds:
    if Mod.value=='Gear Drive':
        W_hover_motor=Wpro_hover*Nred # [rad/s] Nominal motor speed with reduction
        W_cl_motor=Wpro_cl*Nred # [rad/s] Motor Climb speed with reduction
        W_to_motor=Wpro_to*Nred # [rad/s] Motor take-off speed with reduction
    else:
        W_hover_motor=Wpro_hover # [rad/s] Nominal motor speed 
        W_cl_motor=Wpro_cl # [rad/s] Motor Climb speed
        W_to_motor=Wpro_to # [rad/s] Motor take-off speed
        
    #Motor torque:
    
    if Mod.value=='Gear Drive':       
        Tmot_hover=Qpro_hover/Nred # [N.m] motor nominal torque with reduction
        Tmot_to=Qpro_to/Nred # [N.m] motor take-off torque with reduction
        Tmot_cl=Qpro_cl/Nred # [N.m] motor climbing torque with reduction
        

    else:
        Tmot_hover=Qpro_hover# [N.m] motor take-off torque
        Tmot_to=Qpro_to # [N.m] motor take-off torque
        Tmot_cl=Qpro_cl # [N.m] motor climbing torque
        

    Tmot=k_mot*Tmot_hover# [N.m] required motor nominal torque for reductor
    # Selection with take-off speed
    Ktmot=V_bat_est/(k_speed_mot*W_to_motor) # [N.m/A] or [V/(rad/s)] Kt motor (RI term is missing)
    
    if C3.value =='Find a lower ref':
        Tmot=lowerRef('Tnom_Nm',Tmot,'Kt_Nm_A',Ktmot,C4.value,df_mot)[0]
        Ktmot=lowerRef('Tnom_Nm',Tmot,'Kt_Nm_A',Ktmot,C4.value,df_mot)[1]
        modelMot=lowerRef('Tnom_Nm',Tmot,'Kt_Nm_A',Ktmot,C4.value,df_mot)[2].iloc[0]
#         print(modelMot)
        
    elif C3.value =='Find the closest ref':
        Tmot=closeRef('Tnom_Nm',Tmot,'Kt_Nm_A',Ktmot,C4.value,df_mot)[0]
        Ktmot=closeRef('Tnom_Nm',Tmot,'Kt_Nm_A',Ktmot,C4.value,df_mot)[1]
        modelMot=closeRef('Tnom_Nm',Tmot,'Kt_Nm_A',Ktmot,C4.value,df_mot)[2].iloc[0]
#         print(modelMot)
    else:
        Tmot=upperRef('Tnom_Nm',Tmot,'Kt_Nm_A',Ktmot,C4.value,df_mot)[0]
        Ktmot=upperRef('Tnom_Nm',Tmot,'Kt_Nm_A',Ktmot,C4.value,df_mot)[1]
        modelMot=upperRef('Tnom_Nm',Tmot,'Kt_Nm_A',Ktmot,C4.value,df_mot)[2].iloc[0]
#         print(modelMot)

    Tmot_max=Tmot_max_ref*(Tmot/Tmot_ref)**(1) # [N.m] max torque
      
    Mmot=Mmot_ref*(Tmot/Tmot_ref)**(3/3.5) # [kg] Motor mass
    
    Rmot=Rmot_ref*(Tmot/Tmot_ref)**(-5/3.5)*(Ktmot/Ktmot_ref)**(2)  # [Ohm] motor resistance
    Tfmot=Tfmot_ref*(Tmot/Tmot_ref)**(3/3.5) # [N.m] Friction torque

    # Hover current and voltage
    
    Imot_hover = (Tmot_hover+Tfmot)/Ktmot # [I] Current of the motor per propeller
    Umot_hover = Rmot*Imot_hover + W_hover_motor*Ktmot # [V] Voltage of the motor per propeller
    P_el_hover = Umot_hover*Imot_hover # [W] Hover : output electrical power
    
    # Take-Off current and voltage
    Imot_to = (Tmot_to+Tfmot)/Ktmot # [I] Current of the motor per propeller
    Umot_to = Rmot*Imot_to + W_to_motor*Ktmot # [V] Voltage of the motor per propeller
    P_el_to = Umot_to*Imot_to # [W] Takeoff : output electrical power
    
    # Climbing current and voltage
    Imot_cl = (Tmot_cl+Tfmot)/Ktmot # [I] Current of the motor per propeller for climbing
    Umot_cl = Rmot*Imot_cl + W_cl_motor*Ktmot # [V] Voltage of the motor per propeller for climbing
    P_el_cl = Umot_cl*Imot_cl # [W] Power : output electrical power for climbing

    #Gear box model
    if Mod.value=='Gear Drive':
        mg1=0.0309*Nred**2+0.1944*Nred+0.6389       # Ratio input pinion to mating gear
        WF=1+1/mg1+mg1+mg1**2+Nred**2/mg1+Nred**2   # Weight Factor (ƩFd2/C) [-]
        k_sd=1000                                   # Surface durability factor [lb/in]
        C=2*8.85*Tmot_hover/k_sd                    # Coefficient (C=2T/K) [in3]
        Fd2=WF*C                                    # Solid rotor volume [in3]
        Mgear=Fd2*0.3*0.4535                        # Mass reducer [kg] (0.3 is a coefficient evaluated for aircraft application and 0.4535 to pass from lb to kg)
        Fdp2=C*(Nred+1)/Nred                        # Solid rotor pinion volume [in3]
        dp=(Fdp2/0.7)**(1/3)*0.0254                 # Pinion diameter [m] (0.0254 to pass from in to m)
        dg=Nred*dp                                  # Gear diameter [m]
        di=mg1*dp                                   # Inler diameter [m]        
        
# Battery selection & scaling laws sized from hover
# --- 
    # Battery
    # Ref : Prolitex TP3400-4SPX25
    Mbat_ref=.329 # [kg] mass
    #Ebat_ref=4*3.7*3.3*3600 # [J] energy
    #Ebat_ref=220*3600*.329 # [J]
    Cbat_ref= 3.400*3600#[A.s] 
    Vbat_ref=4*3.7#[V] 
    Imax_ref=170#[A]

    Ncel=V_bat_est/3.7# [-] Cell number, round (up value)
    V_bat=3.7*Ncel # [V] Battery voltage
    
    Mbat=k_Mb*eval(M_load.value) # Battery mass
    
    # Hover --> autonomy
    C_bat = Mbat/Mbat_ref*Cbat_ref/V_bat*Vbat_ref # [A.s] Capacity  of the battery 
    
    if C7.value =='Find a lower ref':
        C_bat=C_bat*1000/3600 #[mAh] Capacity battery in mAh
        C_bat=lowerRef('Capacity_mAh',C_bat,'Voltage_V',V_bat,C8.value,df_bat)[0]/1000*3600#[As] Capacity battery As
        V_bat=lowerRef('Capacity_mAh',C_bat,'Voltage_V',V_bat,C8.value,df_bat)[1]
        modelBat=lowerRef('Capacity_mAh',C_bat,'Voltage_V',V_bat,C8.value,df_bat)[2].iloc[0]
    elif C7.value =='Find the closest': 
        C_bat=C_bat*1000/3600 #[mAh] Capacity battery in mAh
        C_bat=closeRef('Capacity_mAh',C_bat,'Voltage_V',V_bat,C8.value,df_bat)[0]/1000*3600#[As] Capacity battery As
        V_bat=closeRef('Capacity_mAh',C_bat,'Voltage_V',V_bat,C8.value,df_bat)[1]
        modelBat=closeRef('Capacity_mAh',C_bat,'Voltage_V',V_bat,C8.value,df_bat)[2].iloc[0]
    else:
        C_bat=C_bat*1000/3600 #[mAh] Capacity battery in mAh
        C_bat=lowerRef('Capacity_mAh',C_bat,'Voltage_V',V_bat,C8.value,df_bat)[0]/1000*3600#[As] Capacity battery As
        V_bat=upperRef('Capacity_mAh',C_bat,'Voltage_V',V_bat,C8.value,df_bat)[1]
        modelBat=upperRef('Capacity_mAh',C_bat,'Voltage_V',V_bat,C8.value,df_bat)[2].iloc[0]

    I_bat = (P_el_hover*Npro)/.95/V_bat # [I] Current of the battery
    t_hf = .8*C_bat/I_bat/60 # [min] Hover time 
    Imax=Imax_ref*C_bat/Cbat_ref # [A] max current battery
    

# ESC sized from max speed
# Ref : Turnigy K_Force 70HV 
    Pesc_ref=3108 # [W] Power
    Vesc_ref=44.4 #[V]Voltage
    Mesc_ref=.115 # [kg] Mass
    
    P_esc=k_ESC*(P_el_to*V_bat/Umot_to) # [W] power electronic power max thrust
    
    P_esc_cl=P_el_cl*V_bat/Umot_cl # [W] power electronic power max climb
    
    Mesc = Mesc_ref*(P_esc/Pesc_ref) # [kg] Mass ESC
    Vesc = Vesc_ref*(P_esc/Pesc_ref)**(1/3)# [V] ESC voltage
    
    if C5.value =='Find a lower ref':
        P_esc=lowerRef('Pmax.in[W]',P_esc,'Vmax.in[V]',Vesc,C6.value,df_esc)[0]
        Vesc=lowerRef('Pmax.in[W]',P_esc,'Vmax.in[V]',Vesc,C6.value,df_esc)[1]
        modelESC=lowerRef('Pmax.in[W]',P_esc,'Vmax.in[V]',Vesc,C6.value,df_esc)[2].iloc[0]
    elif C5.value =='Find the closest ref':
        P_esc=closeRef('Pmax.in[W]',P_esc,'Vmax.in[V]',Vesc,C6.value,df_esc)[0]
        Vesc=closeRef('Pmax.in[W]',P_esc,'Vmax.in[V]',Vesc,C6.value,df_esc)[1]
        modelESC=closeRef('Pmax.in[W]',P_esc,'Vmax.in[V]',Vesc,C6.value,df_esc)[2].iloc[0]
    else:
        P_esc=upperRef('Pmax.in[W]',P_esc,'Vmax.in[V]',Vesc,C6.value,df_esc)[0]
        Vesc=upperRef('Pmax.in[W]',P_esc,'Vmax.in[V]',Vesc,C6.value,df_esc)[1]
        modelESC=upperRef('Pmax.in[W]',P_esc,'Vmax.in[V]',Vesc,C6.value,df_esc)[2].iloc[0]

# Frame sized from max thrust
# ---

    Mfra_ref=.347 #[kg] MK7 frame
    Marm_ref=0.14#[kg] Mass of all arms

    # Length calculation   
#    sep= 2*pi/Narm #[rad] interior angle separation between propellers
    Lbra=Dpro/2/(math.sin(pi/eval(Narm.value)))  #[m] length of the arm

    # Static stress
    Sigma_max=280e6/4 # [Pa] Composite max stress (2 reduction for dynamic, 2 reduction for stress concentration)
    
    # Tube diameter & thickness
    Dout=(F_pro_to*Lbra*32/(pi*Sigma_max*(1-D_ratio**4)))**(1/3) # [m] outer diameter of the beam 
    D_ratio # [m] inner diameter of the beam

    # Mass
    Marm=pi/4*(Dout**2-(D_ratio*Dout)**2)*Lbra*1700*eval(Narm.value) # [kg] mass of the arms

    Mfra=Mfra_ref*(Marm/Marm_ref)# [kg] mass of the frame

    # Thrust Bearing reference
    # Ref : SKF 31309/DF
    Life=5000                                                         # Life time [h]
    k_bear=1
    Cd_bear_ref=2700                                                  # Dynamic reference Load [N]
    C0_bear_ref=1500                                                  # Static reference load[N]
    Db_ref=0.032                                                      # Exterior reference diameter [m]
    Lb_ref=0.007                                                      # Reference lenght [m]
    db_ref=0.020                                                      # Interior reference diametere [m]
    Mbear_ref=0.018                                                   # Reference mass [kg]

    # Thrust bearing model"""
    L10=(60*(Wpro_hover*60/2/3.14)*(Life/10**6))                     # Nominal endurance [Hours of working]
    Cd_ap=(2*F_pro_hov*L10**(1/3))/2                                # Applied load on bearing [N]
    Fmax=2*4*F_pro_to/2
    C0_bear=k_bear*Fmax                                               # Static load [N]
    Cd_bear=Cd_bear_ref/C0_bear_ref**(1.85/2)*C0_bear**(1.85/2)       # Dynamic Load [N]
    Db=Db_ref/C0_bear_ref**0.5*C0_bear**0.5                           # Bearing exterior Diameter [m]
    db=db_ref/C0_bear_ref**0.5*C0_bear**0.5                           # Bearing interior Diameter [m]
    Lb=Lb_ref/C0_bear_ref**0.5*C0_bear**0.5                           # Bearing lenght [m]
    Mbear=Mbear_ref/C0_bear_ref**1.5*C0_bear**1.5                     # Bearing mass [kg]
    
   

    # Objective and Constraints sum up
    # ---
    if Mod.value=='Direct Drive':
        Mtotal_final = (Mesc+Mpro+Mmot+Mbear)*Npro+eval(M_load.value)+Mbat+Mfra+Marm #total mass without reducer
    else:
        Mtotal_final = (Mesc+Mpro+Mmot+Mgear+Mbear)*Npro+eval(M_load.value)+Mbat+Mfra+Marm #total mass with reducer
  
    if Obj.value=='Maximize time':
        constraints = [(Mtotal-Mtotal_final)/Mtotal_final,                
                   (eval(NDmax.value)/60*0.0254-n_pro_cl*Dpro)/(eval(NDmax.value)/60*0.0254),
                   (Tmot_max-Tmot_to)/Tmot_max,
                   (Tmot_max-Tmot_cl)/Tmot_max,
                   (-J*n_pro_cl*Dpro+eval(V_cl.value)),
                   0.01+(J*n_pro_cl*Dpro-eval(V_cl.value)), 
                   (V_bat-Umot_to)/V_bat,
                   (V_bat-Umot_cl)/V_bat,
                   (V_bat-Vesc)/V_bat,
                   (V_bat*Imax-Umot_to*Imot_to*Npro/0.95)/(V_bat*Imax),
                   (V_bat*Imax-Umot_cl*Imot_cl*Npro/0.95)/(V_bat*Imax),
                   (P_esc-P_esc_cl)/P_esc,
                   (eval(MTOW.value)-Mtotal_final)/Mtotal_final
                  ] 
    else:
        constraints = [(Mtotal-Mtotal_final)/Mtotal_final,
                   (eval(NDmax.value)/60*0.0254-n_pro_cl*Dpro)/(eval(NDmax.value)/60*0.0254),
                   (Tmot_max-Tmot_to)/Tmot_max,
                   (Tmot_max-Tmot_cl)/Tmot_max,
                   (-J*n_pro_cl*Dpro+eval(V_cl.value)), 
                   0.01+(J*n_pro_cl*Dpro-eval(V_cl.value)), 
                   (V_bat-Umot_to)/V_bat,
                   (V_bat-Umot_cl)/V_bat,
                   (V_bat-Vesc)/V_bat,
                   (V_bat*Imax-Umot_to*Imot_to*Npro/0.95)/(V_bat*Imax),
                   (V_bat*Imax-Umot_cl*Imot_cl*Npro/0.95)/(V_bat*Imax),
                   (P_esc-P_esc_cl)/P_esc,
                   (t_hf-eval(t_h.value))/t_hf,
                  ] 

    # Objective and contraints
# Objective and contraints
    if arg=='Obj':
        if Obj.value=='Maximize time':
            return 1/t_hf # for time maximisation
        else:
            return Mtotal_final # for mass optimisation 
    if arg=='ObjP':
        P=0 # Penalisation nulle
        if Obj.value=='Minimize mass':
            for C in constraints: 
                if (C<0.): 
                    P=P-1e9*C
            return Mtotal_final+P # for mass optimisation       
        else:
            for C in constraints: 
                if (C<0.): 
                    P=P-1e9*C
            return 1/t_hf+P # for time optimisation       
            
        
    elif arg=='Prt':
        col_names_opt = ['Type', 'Name', 'Min', 'Value', 'Max', 'Unit', 'Comment']

        df_opt = pd.DataFrame()

        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'k_M', 'Min': bounds[0][0], 'Value': k_M, 'Max': bounds[0][1], 'Unit': '[-]', 'Comment': 'over sizing coefficient on the load mass '}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'k_mot', 'Min': bounds[1][0], 'Value': k_mot, 'Max': bounds[1][1], 'Unit': '[-]', 'Comment': 'over sizing coefficient on the motor torque '}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'k_speed_mot', 'Min': bounds[2][0], 'Value': k_speed_mot, 'Max': bounds[2][1], 'Unit': '[-]', 'Comment': 'over sizing coefficient on the motor speed'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'k_vb', 'Min': bounds[3][0], 'Value': k_vb, 'Max': bounds[3][1], 'Unit': '[-]', 'Comment': 'over sizing coefficient for the battery voltage'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'k_ND', 'Min': bounds[4][0], 'Value': k_ND, 'Max': bounds[4][1], 'Unit': '[-]', 'Comment': 'Ratio ND/NDmax'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'D_ratio', 'Min': bounds[5][0], 'Value': D_ratio, 'Max': bounds[5][1], 'Unit': '[-]', 'Comment': 'aspect ratio e/c (thickness/side) for the beam of the frame'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'k_Mb', 'Min': bounds[6][0], 'Value': k_Mb, 'Max': bounds[6][1], 'Unit': '[-]', 'Comment': 'over sizing coefficient on the battery load mass '}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'beta_pro', 'Min': bounds[7][0], 'Value': beta, 'Max': bounds[7][1], 'Unit': '[-]', 'Comment': 'pitch/diameter ratio of the propeller'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'J', 'Min': bounds[8][0], 'Value': J, 'Max': bounds[8][1], 'Unit': '[-]', 'Comment': 'Advance ratio'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'k_ESC', 'Min': bounds[9][0], 'Value': k_ESC, 'Max': bounds[9][1], 'Unit': '[-]', 'Comment': 'over sizing coefficient on the ESC power'}])[col_names_opt]
        if Mod.value=='Gear Drive':       
            df_opt = df_opt.append([{'Type': 'Optimization', 'Name': 'N_red', 'Min': bounds[10][0], 'Value': Nred, 'Max': bounds[10][1], 'Unit': '[-]', 'Comment': 'Reduction ratio'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 0', 'Min': 0, 'Value': constraints[0], 'Max': '-', 'Unit': '[-]', 'Comment': '(Mtotal-Mtotal_final)'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 1', 'Min': 0, 'Value': constraints[1], 'Max': '-', 'Unit': '[-]', 'Comment': '(NDmax-n_pro_cl*Dpro)/NDmax'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 2', 'Min': 0, 'Value': constraints[2], 'Max': '-', 'Unit': '[-]', 'Comment': '(Tmot_max-Tmot_to)/Tmot_max'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 3', 'Min': 0, 'Value': constraints[3], 'Max': '-', 'Unit': '[-]', 'Comment': '(Tmot_max-Tmot_cl)/Tmot_max'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 4', 'Min': 0, 'Value': constraints[4], 'Max': '-', 'Unit': '[-]', 'Comment': '(-J*n_pro_cl*Dpro+V_cl)'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 5', 'Min': 0, 'Value': constraints[5], 'Max': '-', 'Unit': '[-]', 'Comment': '0.01+(+J*n_pro_cl*Dpro-V_cl)'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 6', 'Min': 0, 'Value': constraints[6], 'Max': '-', 'Unit': '[-]', 'Comment': '(V_bat-Umot_to)/V_bat'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 7', 'Min': 0, 'Value': constraints[7], 'Max': '-', 'Unit': '[-]', 'Comment': '(V_bat-Umot_cl)/V_bat'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 8', 'Min': 0, 'Value': constraints[8], 'Max': '-', 'Unit': '[-]', 'Comment': '(V_bat-Vesc)/V_bat'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 9', 'Min': 0, 'Value': constraints[9], 'Max': '-', 'Unit': '[-]', 'Comment': '(V_bat*Imax-Umot_to*Imot_to*Npro/0.95)/(V_bat*Imax)'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 10', 'Min': 0, 'Value': constraints[10], 'Max': '-', 'Unit': '[-]', 'Comment': '(V_bat*Imax-Umot_cl*Imot_cl*Npro/0.95)/(V_bat*Imax)'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 11', 'Min': 0, 'Value': constraints[11], 'Max': '-', 'Unit': '[-]', 'Comment': '(P_esc-P_esc_cl)/P_esc'}])[col_names_opt]
        if Obj.value=='Minimize mass':
            df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 12', 'Min': 0, 'Value': constraints[12], 'Max': '-', 'Unit': '[-]', 'Comment': '(t_hf-t_h)/t_hf'}])[col_names_opt]
        else:
            df_opt = df_opt.append([{'Type': 'Constraints', 'Name': 'Const 12', 'Min': 0, 'Value': constraints[12], 'Max': '-', 'Unit': '[-]', 'Comment': '(MTOW-Mtotal_final)/Mtotal_final'}])[col_names_opt]
        df_opt = df_opt.append([{'Type': 'Objective', 'Name': 'Objective', 'Min': 0, 'Value': Mtotal_final, 'Max': '-', 'Unit': '[kg]', 'Comment': 'Total mass'}])[col_names_opt]


        col_names = ['Type', 'Name', 'Value', 'Unit', 'Comment']

        df = pd.DataFrame()

        df = df.append([{'Type': 'Propeller', 'Name': modelPro, 'Value': '-','Unit': '-', 'Comment': 'Propeller model'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'F_pro_to', 'Value': F_pro_to, 'Unit': '[N]', 'Comment': 'Thrust for 1 propeller during Take Off'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'F_pro_cl', 'Value': F_pro_cl, 'Unit': '[N]', 'Comment': 'Thrust for 1 propeller during Take Off'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'F_pro_hov', 'Value': F_pro_hov, 'Unit': '[N]', 'Comment': 'Thrust for 1 propeller during Hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'rho_air', 'Value': eval(rho_air.value), 'Unit': '[kg/m^3]', 'Comment': 'Air density'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'ND_max', 'Value': eval(NDmax.value), 'Unit': '[RPM*in]', 'Comment': 'Max speed limit (N.D max)'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'Dpro_ref', 'Value': Dpro_ref, 'Unit': '[m]', 'Comment': 'Reference propeller diameter'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'M_pro_ref', 'Value': Mpro_ref, 'Unit': '[kg]', 'Comment': 'Reference propeller mass'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'C_t_sta', 'Value': C_t_sta, 'Unit': '[-]', 'Comment': 'Static thrust coefficient of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'C_t_dyn', 'Value': C_t_dyn, 'Unit': '[-]', 'Comment': 'Dynamic thrust coefficient of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'C_p_sta', 'Value': C_p_sta, 'Unit': '[-]', 'Comment': 'Static power coefficient of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'C_p_dyn', 'Value': C_p_dyn, 'Unit': '[-]', 'Comment': 'Dynamic power coefficient of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'D_pro', 'Value': Dpro, 'Unit': '[m]', 'Comment': 'Diameter of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'n_pro_cl', 'Value': n_pro_cl, 'Unit': '[Hz]', 'Comment': 'Rev speed of the propeller during climbing'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'n_pro_to', 'Value': n_pro_to, 'Unit': '[Hz]', 'Comment': 'Rev speed of the propeller during takeoff'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'n_pro_hov', 'Value': n_pro_hover, 'Unit': '[Hz]', 'Comment': 'Rev speed of the propeller during hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'P_pro_cl', 'Value': Ppro_cl, 'Unit': '[W]', 'Comment': 'Power on the mechanical shaft of the propeller during climbing'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'P_pro_to', 'Value': Ppro_to, 'Unit': '[W]', 'Comment': 'Power on the mechanical shaft of the propeller during takeoff'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'P_pro_hov', 'Value': Ppro_hover, 'Unit': '[W]', 'Comment': 'Power on the mechanical shaft of the propeller during hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'M_pro', 'Value': Mpro, 'Unit': '[kg]', 'Comment': 'Mass of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'Omega_pro_cl', 'Value': Wpro_cl, 'Unit': '[rad/s]', 'Comment': 'Rev speed of the propeller during climbing'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'Omega_pro_to', 'Value': Wpro_to, 'Unit': '[rad/s]', 'Comment': 'Rev speed of the propeller during takeoff'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'Omega_pro_hov', 'Value': Wpro_hover, 'Unit': '[rad/s]', 'Comment': 'Rev speed of the propeller during hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'T_pro_hov', 'Value': Qpro_hover, 'Unit': '[N.m]', 'Comment': 'Torque on the mechanical shaft of the propeller during hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'T_pro_to', 'Value': Qpro_to, 'Unit': '[N.m]', 'Comment': 'Torque on the mechanical shaft of the propeller during takeoff'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'T_pro_cl', 'Value': Qpro_cl, 'Unit': '[N.m]', 'Comment': 'Torque on the mechanical shaft of the propeller during climbing'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': modelMot, 'Value': '-','Unit': '-', 'Comment': 'Motor model'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'T_max_mot_ref', 'Value': Tmot_max_ref, 'Unit': '[N.m]', 'Comment': 'Max torque'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'R_mot_ref', 'Value': Rmot_ref, 'Unit': '[Ohm]', 'Comment': 'Resistance'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'M_mot_ref', 'Value': Mmot_ref, 'Unit': '[kg]', 'Comment': 'Reference motor mass'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'K_mot_ref', 'Value': Ktmot_ref, 'Unit': '[N.m/A]', 'Comment': 'Torque coefficient'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'T_mot_fr_ref', 'Value': Tfmot_ref, 'Unit': '[N.m]', 'Comment': 'Friction torque (zero load, nominal speed)'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'T_nom_mot', 'Value': Tmot_hover, 'Unit': '[N.m]', 'Comment': 'Continuous of the selected motor torque'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'T_mot_to', 'Value': Tmot_to, 'Unit': '[N.m]', 'Comment': 'Transient torque possible for takeoff'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'T_mot_cl', 'Value': Tmot_cl, 'Unit': '[N.m]', 'Comment': 'Transient torque possible for climbing'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'R_mot', 'Value': Rmot, 'Unit': '[Ohm]', 'Comment': 'Resistance'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'M_mot', 'Value': Mmot, 'Unit': '[kg]', 'Comment': 'Motor mass'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'K_mot', 'Value': Ktmot, 'Unit': '[N.m/A', 'Comment': 'Torque constant of the selected motor'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'T_mot_fr', 'Value': Tfmot, 'Unit': '[N.m]', 'Comment': 'Friction torque of the selected motor'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'I_mot_hov', 'Value': Imot_hover, 'Unit': '[A]', 'Comment': 'Motor current for hover'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'I_mot_to', 'Value': Imot_to, 'Unit': '[A]', 'Comment': 'Motor current for takeoff'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'I_mot_cl', 'Value': Imot_cl, 'Unit': '[A]', 'Comment': 'Motor current for climbing'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'U_mot_cl', 'Value': Umot_hover, 'Unit': '[V]', 'Comment': 'Motor voltage for climbing'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'U_mot_to', 'Value': Umot_to, 'Unit': '[V]', 'Comment': 'Motor voltage for takeoff'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'U_mot', 'Value': Umot_hover, 'Unit': '[V]', 'Comment': 'Nominal voltage '}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'P_el_mot_cl', 'Value': P_el_cl, 'Unit': '[W]', 'Comment': 'Motor electrical power for climbing'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'P_el_mot_to', 'Value': P_el_to, 'Unit': '[W]', 'Comment': 'Motor electrical power for takeoff'}])[col_names]
        df = df.append([{'Type': 'Motor', 'Name': 'P_el_mot_hov', 'Value': P_el_hover, 'Unit': '[W]', 'Comment': 'Motor electrical power for hover'}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': modelBat, 'Value': '-','Unit': '-', 'Comment': 'Battery model'}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'M_bat_ref', 'Value': Mbat_ref, 'Unit': '[kg]', 'Comment': 'Mass of the reference battery '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'M_esc_ref', 'Value': Mesc_ref, 'Unit': '[kg]', 'Comment': 'Reference ESC mass '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'P_esc_ref', 'Value': Pesc_ref, 'Unit': '[W]', 'Comment': 'Reference ESC power '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'N_s_bat', 'Value': np.ceil(Ncel), 'Unit': '[-]', 'Comment': 'Number of battery cells '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'U_bat', 'Value': V_bat, 'Unit': '[V]', 'Comment': 'Battery voltage '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'M_bat', 'Value': Mbat, 'Unit': '[kg]', 'Comment': 'Battery mass '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'C_bat', 'Value': C_bat, 'Unit': '[A.s]', 'Comment': 'Battery capacity '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'I_bat', 'Value': I_bat, 'Unit': '[A]', 'Comment': 'Battery current '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 't_hf', 'Value': t_hf, 'Unit': '[min]', 'Comment': 'Hovering time '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': modelESC, 'Value': '-','Unit': '-', 'Comment': 'ESC model'}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'P_esc', 'Value': P_esc, 'Unit': '[W]', 'Comment': 'Power electronic power (corner power or apparent power) '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'M_esc', 'Value': Mesc, 'Unit': '[kg]', 'Comment': 'ESC mass '}])[col_names]
        df = df.append([{'Type': 'Battery & ESC', 'Name': 'V_esc', 'Value': Vesc, 'Unit': '[V]', 'Comment': 'ESC voltage '}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'N_arm', 'Value': eval(Narm.value), 'Unit': '[-]', 'Comment': 'Number of arms '}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'N_pro_arm', 'Value': eval(Np_arm.value), 'Unit': '[-]', 'Comment': 'Number of propellers per arm '}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'sigma_max', 'Value': Sigma_max, 'Unit': '[Pa]', 'Comment': 'Max admisible stress'}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'L_arm', 'Value': Lbra, 'Unit': '[m]', 'Comment': 'Length of the arm'}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'D_out', 'Value': Dout, 'Unit': '[m]', 'Comment': 'Outer diameter of the arm (tube)'}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'Marm', 'Value': Marm, 'Unit': '[kg]', 'Comment': '1 Arm mass'}])[col_names]
        df = df.append([{'Type': 'Frame', 'Name': 'M_frame', 'Value': Mfra, 'Unit': '[kg]', 'Comment': 'Frame mass'}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 'M_load', 'Value': eval(M_load.value), 'Unit': '[kg]', 'Comment': 'Payload mass'}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 't_hf', 'Value': eval(t_h.value), 'Unit': '[min]', 'Comment': 'Hovering time '}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 'k_maxthrust', 'Value': eval(k_maxthrust.value), 'Unit': '[-]', 'Comment': 'Ratio max thrust'}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 'N_arm', 'Value': eval(Narm.value), 'Unit': '[-]', 'Comment': 'Number of arms '}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 'N_pro_arm', 'Value': eval(Np_arm.value), 'Unit': '[-]', 'Comment': 'Number of propellers per arm '}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 'V_cl', 'Value': eval(V_cl.value), 'Unit': '[m/s]', 'Comment': 'Climb speed'}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 'CD', 'Value': eval(C_D.value), 'Unit': '[-]', 'Comment': 'Drag coefficient'}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 'A_top', 'Value': eval(A_top.value), 'Unit': '[m^2]', 'Comment': 'Top surface'}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 'MTOW', 'Value': eval(MTOW.value), 'Unit': '[kg]', 'Comment': 'Max takeoff Weight'}])[col_names]
        df = df.append([{'Type': 'Specifications', 'Name': 'NDmax', 'Value': eval(NDmax.value), 'Unit': '[RPM*inch]', 'Comment': 'RPM limit APC MR'}])[col_names]

        items = sorted(df['Type'].unique().tolist())+['Optimization']
        return df, df_opt 
        
    else:
        return constraints

### 4.-Optimization variables

In [5]:
# Optimisation variables
k_M=3.2              # over sizing coefficient on the load mass 
k_mot=1.1               # over sizing coefficient on the motor torque
k_speed_mot=1.2      # adaption winding coef on the motor speed
k_vb=1.1              # oversizing coefficient for voltage evaluation
k_ND=0.8               # reduction of product speed.diameter on the propeller
D_ratio=.02          # aspect ratio e/c (thickness/side) for the beam of the frame
k_Mb=1.1                # ratio battery/load mass
beta=.33             # pitch/diameter ratio of the propeller
J=0.2                # advance ratio [-]
k_ESC=1.1              # oversizing coefficient on the ESC power
Nred=1.1                # Reduction ratio

    
bounds=[(1,400),#k_M
       (1,20),#k_mot
       (1,10),#k_speed_mot
       (1,5),#k_vb
       (0.01,1),#k_ND
       (0.05,.99),#D_ratio
       (.01,60),#k_Mb
       (0.3,0.6),#beta
       (0.01,0.5),#J
       (1,15),#k_ESC
       (1,20),#Nred
    ]

### 5.-Result

In [6]:
from IPython.display import display, clear_output
from ipywidgets import widgets
button = widgets.Button(description="Calculate")
display(button)

# df_pro_mod=df_pro[df_pro['TYPE']==pro.value]
# df_bat_mod=df_bat[df_bat['TYPE']==bat.value]
# df_esc_mod=df_esc[df_esc['TYPE']==esc.value]
# df_mot_mod=df_mot[df_mot['TYPE']==mot.value]

output = widgets.Output()

@output.capture()
def on_button_clicked(b):
    clear_output()    
    
    # Vector of parameters
    if Mod.value=='Gear Drive':       
        parameters = scipy.array((k_M,k_mot,k_speed_mot,k_vb,k_ND, D_ratio, k_Mb, beta, J, k_ESC, Nred))
    else:
        parameters = scipy.array((k_M,k_mot,k_speed_mot,k_vb,k_ND, D_ratio, k_Mb, beta, J, k_ESC))
    print(parameters)
    # optimization with SLSQP algorithm
    contrainte=lambda x: SizingCode(x, 'Const')
    objectif=lambda x: SizingCode(x, 'Obj')
    objectifP=lambda x: SizingCode(x, 'ObjP')


    # Differential evolution omptimisation
    start = time.time()
    print("Loading")

    if Algo.value=='SLSQP':
        if Mod.value=='Gear Drive':       
            result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, 
                                       bounds=[(1,400),#k_M
                                               (1,20),#k_mot
                                               (1,10),#k_speed_mot
                                               (1,5),#k_vb
                                               (0.001,1),#k_ND
                                               (0.05,.99),#D_ratio
                                               (.001,60),#k_Mb
                                               (0.3,0.6),#beta
                                               (0.001,0.5),#J
                                               (1,15),#k_ESC
                                               (1,20)]#Nred
                                              ,f_ieqcons=contrainte, iter=1500, acc=1e-12)
        else:
            result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, 
                                       bounds=[(1,400),#k_M
                                               (1,20),#k_mot
                                               (1,10),#k_speed_mot
                                               (1,5),#k_vb
                                               (0.001,1),#k_ND
                                               (0.05,.99),#D_ratio
                                               (.001,60),#k_Mb
                                               (0.3,0.6),#beta
                                               (0.001,0.5),#J
                                               (1,15)]#k_ESC
                                              ,f_ieqcons=contrainte, iter=1500, acc=1e-12)
    else:
        if Mod.value=='Gear Drive':       
            result = scipy.optimize.differential_evolution(func=objectifP,
                                   bounds=[(1,400),#k_M
                                           (1,20),#k_mot
                                           (1,10),#k_speed_mot
                                           (1,5),#k_vb
                                           (0.001,1),#k_ND
                                           (0.05,.99),#D_ratio
                                           (.001,60),#k_Mb
                                           (0.3,0.6),#beta
                                           (0.0001,0.5),#J
                                           (1,15),#k_ESC
                                           (1,20),#Nred
                                        ],maxiter=100,
                                   tol=1e-12)
        else:
            result = scipy.optimize.differential_evolution(func=objectifP,
                                   bounds=[(1,400),#k_M
                                           (1,20),#k_mot
                                           (1,10),#k_speed_mot
                                           (1,5),#k_vb
                                           (0.001,1),#k_ND
                                           (0.05,.99),#D_ratio
                                           (.001,60),#k_Mb
                                           (0.3,0.6),#beta
                                           (0.0001,0.5),#J
                                           (1,15),#k_ESC
                                        ],maxiter=100,
                                   tol=1e-12)
                                               
    print("end")
    # Final characteristics after optimization 
    end = time.time()
    print("Operation time: %.5f s" %(end - start))

    print("-----------------------------------------------")
    print("Final characteristics after optimization :")

    if Algo.value=='SLSQP':
        data=SizingCode(result, 'Prt')[0]
        data_opt=SizingCode(result, 'Prt')[1]
    else:
        data=SizingCode(result.x, 'Prt')[0]
        data_opt=SizingCode(result.x, 'Prt')[1]
    print("-----------------------------------------------")


    pd.options.display.float_format = '{:,.3f}'.format


    def view(x=''):
        #if x=='All': return display(df)
        if x=='Optimization' : return display(data_opt)
        return display(data[data['Type']==x])

    items = sorted(data['Type'].unique().tolist())+['Optimization']

    w = widgets.Select(options=items)
    return display(interactive(view, x=w))
#     display(data)
button.on_click(on_button_clicked)
display(output)



Button(description='Calculate', style=ButtonStyle())

Output()